In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Create synthetic data
num_users = 1000
num_items = 500
num_samples = 10000

# Random user-item interactions
np.random.seed(0)
user_ids = np.random.randint(0, num_users, num_samples)
item_ids = np.random.randint(0, num_items, num_samples)
ratings = np.random.rand(num_samples)  # Ratings between 0 and 1

# Create DataFrame
data = pd.DataFrame({'user_id': user_ids, 'item_id': item_ids, 'rating': ratings})

# Prepare inputs and labels
X = np.array([data['user_id'].values, data['item_id'].values]).T
y = data['rating'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Separate the features into user_id and item_id
X_train_user = X_train[:, 0]
X_train_item = X_train[:, 1]
X_test_user = X_test[:, 0]
X_test_item = X_test[:, 1]

# Define the recommendation model
def create_recommendation_model(num_users, num_items, embedding_dim=10):
    user_input = Input(shape=(1,))
    item_input = Input(shape=(1,))

    user_embedding = Embedding(num_users, embedding_dim)(user_input)
    item_embedding = Embedding(num_items, embedding_dim)(item_input)

    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)

    concat = Concatenate()([user_vecs, item_vecs])

    x = Dense(64, activation='relu')(concat)
    x = Dense(32, activation='relu')(x)
    rating_output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[user_input, item_input], outputs=rating_output)
    model.compile(optimizer=Adam(), loss='mean_squared_error')

    return model

# Create and train the model
model = create_recommendation_model(num_users, num_items, embedding_dim=10)
model.summary()
x_train=[X_train_user, X_train_item]
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Predict ratings
def predict_rating(user_id, item_id):
    return model.predict([np.array([user_id]), np.array([item_id])])[0][0]

# Example prediction
user_id = 0
item_id = 100
predicted_rating = predict_rating(user_id, item_id)
print(f"Predicted rating for user {user_id} and item {item_id}: {predicted_rating:.2f}")


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 1, 10)          │         10,000 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_5 (Embedding)   │ (None, 1, 10)          │          5,000 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_4 (Flatten)       │ (None, 10)             │              0 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_5 (Flatten)       │ (None, 10)             │              0 │ embedding_5[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 20)             │              0 │ flatten_4[0][0],       │
│ (Concatenate)             │                        │                │ flatten_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │          1,344 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 32)             │          2,080 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 1)              │             33 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 18,457 (72.10 KB)

 Trainable params: 18,457 (72.10 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0831 - val_loss: 0.0857
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0816 - val_loss: 0.0882
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0757 - val_loss: 0.0910
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0678 - val_loss: 0.0958
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0602 - val_loss: 0.1031
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0520 - val_loss: 0.1072
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0420 - val_loss: 0.1152
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0340 - val_loss: 0.1211
Epoch 9/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0265 - val_loss: 0.1296
Epoch 10/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0222 - val_loss: 0.1348
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Predicted rating for user 0 and item 100: 0.11
